<a href="https://colab.research.google.com/github/ge28yen/Weather_prediction/blob/main/Weather_Prediction_mine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Load (and preprocess) the data

In [ ]:
!pip install xarray gcsfs torch zarr
import zarr
import xarray as xr
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import gcsfs  # for accessing data on Google Cloud Storage
import wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 0mche0 (0mche0-my-orga) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Connect to GCS
fs = gcsfs.GCSFileSystem(token="anon")

# Path to the public ERA5 single-level Zarr store
zarr_path = 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'

# Open the dataset (this might take a moment to initialize)
ds = xr.open_zarr(
    zarr_path,
    chunks=None,
    storage_options=dict(token='anon'),
)

# Check available variables
print(ds.data_vars)

Data variables:
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    10m_wind_gust_since_previous_post_processing                     (time, latitude, longitude) float32 5TB ...
    2m_dewpoint_temperature                                          (time, latitude, longitude) float32 5TB ...
    2m_temperature                                                   (time, lati

In [ ]:
surface_vars = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'surface_pressure']
vertical_vars = ['temperature', 'u_component_of_wind', 'v_component_of_wind']
desired_levels = [875, 750, 500, 300, 200, 100]

time_slice = slice("2020-01-01", "2020-01-08")
lat_slice  = slice(50, 40)    # from 50°N down to 40°N
lon_slice  = slice(0, 10)     # from 0°E to 10°E

surf_ds = ds[surface_vars].sel(time=time_slice, latitude=lat_slice, longitude=lon_slice)

vert_ds = ds[vertical_vars].sel(time=time_slice, level=desired_levels, latitude=lat_slice, longitude=lon_slice)

# 4. Normalize and stack the data for each branch
# --- Surface Variables ---
surf_list = []
for var in surface_vars:
    da = surf_ds[var]
    mean_val = da.mean().values

    std_val  = da.std().values
    print(f'variable, {var}, mean: {mean_val}, std: {std_val}')
    da_norm = (da - mean_val) / std_val
    # shape: (time, lat, lon)
    arr = da_norm.values
    surf_list.append(arr)
# Stack along a new channel axis: result shape -> (time, num_surface_vars, lat, lon)
surf_array = np.stack(surf_list, axis=1)
print("Surface array shape:", surf_array.shape)

# --- Vertical Variables ---
vert_list = []
for var in vertical_vars:
    da = vert_ds[var]
    mean_val = da.mean().values
    std_val  = da.std().values
    print(f'variable, {var}, mean: {mean_val}, std: {std_val}')
    da_norm = (da - mean_val) / std_val
    # Convert to NumPy; shape: (time, level, lat, lon)
    arr = da_norm.values
    # Add a channel dimension for this variable so that each variable becomes (time, 1, level, lat, lon)
    arr = np.expand_dims(arr, axis=1)
    vert_list.append(arr)
# Concatenate along the channel axis: result shape -> (time, num_vertical_vars, level, lat, lon)
vert_array = np.concatenate(vert_list, axis=1)
print("Vertical array shape:", vert_array.shape)
# Expected: (time, 3, 2, lat, lon) if there are 3 vertical variables and 2 levels.


variable, 2m_temperature, mean: 279.5635070800781, std: 5.683135032653809
variable, 10m_u_component_of_wind, mean: 0.8511970043182373, std: 2.4158496856689453
variable, 10m_v_component_of_wind, mean: -0.007941002026200294, std: 2.7745039463043213
variable, surface_pressure, mean: 98583.03125, std: 5226.388671875
Surface array shape: (192, 4, 41, 41)
variable, temperature, mean: 243.93789672851562, std: 28.478496551513672
variable, u_component_of_wind, mean: 4.6035380363464355, std: 9.100003242492676
variable, v_component_of_wind, mean: -8.164691925048828, std: 10.139703750610352
Vertical array shape: (192, 3, 6, 41, 41)


In [ ]:

class PanguEraDataset(Dataset):
    def __init__(self, vertical_data, surface_data, seq_length=6, lead_time=1):
        """
        vertical_data: NumPy array with shape (time, channels_v, levels, lat, lon)
        surface_data: NumPy array with shape (time, channels_s, lat, lon)
        seq_length: number of consecutive time steps for the input sequence.
        lead_time: how many time steps ahead for the target (default=1 for single-step forecasting).
        """
        assert vertical_data.shape[0] == surface_data.shape[0], "Time dimensions must match"
        self.vertical = vertical_data
        self.surface  = surface_data
        self.seq_length = seq_length
        self.lead_time  = lead_time

        self.T = vertical_data.shape[0]
        self.num_samples = self.T - self.seq_length - self.lead_time + 1

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        vert_input = self.vertical[idx : idx + self.seq_length]   # shape: (seq_length, channels_v, levels, lat, lon)
        surf_input = self.surface[idx : idx + self.seq_length]      # shape: (seq_length, channels_s, lat, lon)

        vert_target = self.vertical[idx + self.seq_length + (self.lead_time - 1)]  # shape: (channels_v, levels, lat, lon)
        surf_target = self.surface[idx + self.seq_length + (self.lead_time - 1)]     # shape: (channels_s, lat, lon)

        vert_input = torch.tensor(vert_input, dtype=torch.float32)[:, :, :, :40, :40]
        surf_input = torch.tensor(surf_input, dtype=torch.float32)[:, :, :40, :40]
        vert_target = torch.tensor(vert_target, dtype=torch.float32)[:, :, :40, :40]
        surf_target = torch.tensor(surf_target, dtype=torch.float32)[ :, :40, :40]

        return (vert_input, surf_input), (vert_target, surf_target)

dataset = PanguEraDataset(vertical_data=vert_array, surface_data=surf_array, seq_length=6)
print("Dataset length:", len(dataset))

(sample_vert, sample_surf), (target_vert, target_surf) = dataset[0]
print("Vertical input shape:", sample_vert.shape)  # (seq_length, channels_v, levels, lat, lon)
print("Surface input shape:", sample_surf.shape)    # (seq_length, channels_s, lat, lon)
print("Vertical target shape:", target_vert.shape)  # (channels_v, levels, lat, lon)
print("Surface target shape:", target_surf.shape)   # (channels_s, lat, lon)

Dataset length: 186
Vertical input shape: torch.Size([6, 3, 6, 40, 40])
Surface input shape: torch.Size([6, 4, 40, 40])
Vertical target shape: torch.Size([3, 6, 40, 40])
Surface target shape: torch.Size([4, 40, 40])


## 2. Define the architecture

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


def pad3D(x: torch.Tensor, pad=(0,0,0,0,0,0)):
    """ No-op or minimal 3D padding. """
    # In real usage, you'd do e.g. F.pad(x, pad).
    return x

def pad2D(x: torch.Tensor, pad=(0,0,0,0)):
    """ No-op or minimal 2D padding. """
    return x

def crop3D(x: torch.Tensor, crop=(0,0,0,0,0,0)):
    """ No-op 3D cropping. """
    return x

def crop2D(x: torch.Tensor, crop=(0,0,0,0)):
    """ No-op 2D cropping. """
    return x



class PatchEmbedding(nn.Module):
    def __init__(self, patch_size=(2, 2, 2), dim=32, in_channels_3d=5, in_channels_2d=4):
        super().__init__()
        self.patch_size = patch_size
        # 3D conv for upper-air fields
        self.conv3d = nn.Conv3d(
            in_channels=in_channels_3d,
            out_channels=dim,
            kernel_size=patch_size,
            stride=patch_size
        )
        # 2D conv for surface fields
        # We'll artificially add 3 extra channels for masks, etc. as needed, but let's skip that for MVP.
        self.conv2d = nn.Conv2d(
            in_channels=in_channels_2d,
            out_channels=dim,
            kernel_size=patch_size[1:],  # e.g. (2,2)
            stride=patch_size[1:]
        )

    def forward(self, input_3d, input_2d):
        """
        input_3d: (B, in_channels_3d, Z, H, W)
        input_2d: (B, in_channels_2d, H, W)
        Returns: (B, N, dim)
        """
        # Minimal pad
        x3d = pad3D(input_3d)
        x2d = pad2D(input_2d)

        # 3D and 2D projection
        x3d = self.conv3d(x3d)   # (B, dim, Z//pZ, H//pH, W//pW)
        x2d = self.conv2d(x2d)   # (B, dim, H//pH, W//pW)

        # For simplicity, treat x2d as if it were an extra Z channel of size 1:
        B, dim_, z_, h_, w_ = x3d.shape  # note: z_ = Z//pZ, etc.
        # Make x2d -> (B, dim, 1, h_, w_)
        x2d = x2d.unsqueeze(2)

        # Concat along Z-dimension => shape (B, dim, z_+1, h_, w_)
        x = torch.cat([x3d, x2d], dim=2)

        # Flatten to (B, (z_+1)*h_*w_, dim)
        # Current shape: (B, dim, Z+1, H', W')
        x = x.permute(0, 2, 3, 4, 1)        # => (B, Z+1, H', W', dim)
        x = x.reshape(B, -1, dim_)         # => (B, N, dim)
        return x

class PatchRecovery(nn.Module):
    def __init__(self, patch_size=(2,2,2), dim=32, out_channels_3d=5, out_channels_2d=4):
        super().__init__()
        self.patch_size = patch_size
        self.dim = dim
        self.deconv3d = nn.ConvTranspose3d(
            in_channels=dim,
            out_channels=out_channels_3d,
            kernel_size=patch_size,
            stride=patch_size
        )
        self.deconv2d = nn.ConvTranspose2d(
            in_channels=dim,
            out_channels=out_channels_2d,
            kernel_size=patch_size[1:],
            stride=patch_size[1:]
        )

    def forward(self, x, Z, H, W):
        """
        x shape: (B, N, dim)
        Suppose N = (Z//pZ)*(H//pH)*(W//pW) + (H//pH)*(W//pW)
        So we reshape into (B, dim, Z'+1, H', W') => split last Z-slice for surface
        """
        B, N, dim_ = x.shape
        # Hardcode the "patch divisions" for MVP
        pZ, pH, pW = self.patch_size
        Z_ = Z // pZ
        H_ = H // pH
        W_ = W // pW

        # Reshape to (B, Z_+1, H_, W_, dim)
        x = x.view(B, (Z_+1), H_, W_, dim_).permute(0, 4, 1, 2, 3)  # => (B, dim, Z_+1, H_, W_)

        # Split
        x3d = x[:, :, :Z_, :, :]      # (B, dim, Z_, H_, W_)
        x2d = x[:, :,  Z_, :, :]      # (B, dim, H_, W_)

        # Deconvs
        out_3d = self.deconv3d(x3d)   # => (B, out_channels_3d, Z, H, W)
        out_2d = self.deconv2d(x2d)   # => (B, out_channels_2d, H, W)

        # Crop if needed
        out_3d = crop3D(out_3d)
        out_2d = crop2D(out_2d)
        return out_3d, out_2d

class AttentionBlock(nn.Module):
    """
    Minimal multi-head self-attention block on flattened tokens (B, N, dim).
    """
    def __init__(self, dim=32, num_heads=4, attn_drop=0.0, proj_drop=0.0):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        # x: (B, N, dim)
        B, N, C = x.shape
        qkv = self.qkv(x)  # => (B, N, 3*dim)
        qkv = qkv.reshape(B, N, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # => (3, B, heads, N, head_dim)
        q, k, v = qkv[0], qkv[1], qkv[2]  # each => (B, heads, N, head_dim)

        q = q * self.scale
        attn = torch.matmul(q, k.transpose(-2, -1))  # => (B, heads, N, N)
        attn = F.softmax(attn, dim=-1)
        attn = self.attn_drop(attn)

        x_out = torch.matmul(attn, v)  # => (B, heads, N, head_dim)
        x_out = x_out.permute(0, 2, 1, 3).contiguous()
        x_out = x_out.view(B, N, C)  # => (B, N, dim)

        x_out = self.proj(x_out)
        x_out = self.proj_drop(x_out)
        return x_out

class TransformerBlock(nn.Module):
    """
    Minimal block with (attn -> MLP), each with skip-connection + LayerNorm.
    """
    def __init__(self, dim=32, num_heads=4, drop=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = AttentionBlock(dim=dim, num_heads=num_heads)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, dim*4),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(dim*4, dim),
            nn.Dropout(drop),
        )

    def forward(self, x):
        # x: (B, N, dim)
        shortcut = x
        x = self.norm1(x)
        x = self.attn(x)
        x = shortcut + x  # skip

        shortcut = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = shortcut + x
        return x

class MVPModel(nn.Module):
    def __init__(self,
                 patch_size=(2,2,2),
                 in_channels_3d=5,
                 in_channels_2d=4,
                 out_channels_3d=5,
                 out_channels_2d=4,
                 emb_dim=32,
                 num_heads=4):
        super().__init__()
        self.patch_size = patch_size
        self.emb_dim = emb_dim
        self.in_channels_3d = in_channels_3d
        self.in_channels_2d = in_channels_2d
        self.out_channels_3d = out_channels_3d
        self.out_channels_2d = out_channels_2d

        # Patch embedding
        self.embed = PatchEmbedding(
            patch_size=patch_size,
            dim=emb_dim,
            in_channels_3d=in_channels_3d,
            in_channels_2d=in_channels_2d
        )

        # One or two transformer blocks
        self.block1 = TransformerBlock(dim=emb_dim, num_heads=num_heads)
        self.block2 = TransformerBlock(dim=emb_dim, num_heads=num_heads)

        # Patch Recovery
        self.recover = PatchRecovery(
            patch_size=patch_size,
            dim=emb_dim,
            out_channels_3d=out_channels_3d,
            out_channels_2d=out_channels_2d
        )

    def forward(self, input_3d, input_2d):
        """
        input_3d: (B, in_channels_3d, Z, H, W)
        input_2d: (B, in_channels_2d, H, W)
        """
        x = self.embed(input_3d, input_2d)  # => (B, N, emb_dim)

        x = self.block1(x)
        x = self.block2(x)

        # 3. Recover
        B, C3d, Z, H, W = input_3d.shape
        out_3d, out_2d = self.recover(x, Z, H, W)
        return out_3d, out_2d

### Test the dataset on the modle

In [ ]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = MVPModel(
    patch_size=(2, 2, 2),
    in_channels_3d=vert_array.shape[1],    # should be 3 for vertical
    in_channels_2d=surf_array.shape[1],      # should be 4 for surface
    out_channels_3d=vert_array.shape[1],
    out_channels_2d=surf_array.shape[1],
    emb_dim=32,
    num_heads=4
)

model.eval()

for batch in dataloader:

    (vert_seq, surf_seq), (vert_target, surf_target) = batch

    print("Vertical sequence shape:", vert_seq.shape)  # Expected: (B, seq_length, channels_v, levels, H, W)
    print("Surface sequence shape:", surf_seq.shape)    # Expected: (B, seq_length, channels_s, H, W)

    # For a simple forward pass, select the last time step of each sequence.
    # This is a common strategy in forecasting if you use the latest state as input.
    vert_input = vert_seq[:, -1]  # Shape: (B, channels_v, levels, H, W)
    surf_input = surf_seq[:, -1]  # Shape: (B, channels_s, H, W)

    # Pass the inputs through the model
    out_3d, out_2d = model(vert_input, surf_input)

    print("Output vertical branch shape:", out_3d.shape)  # Expected: (B, out_channels_3d, Z, H, W)
    print("Output surface branch shape:", out_2d.shape)    # Expected: (B, out_channels_2d, H, W)
    break  # test one batch only

Vertical sequence shape: torch.Size([2, 6, 3, 6, 40, 40])
Surface sequence shape: torch.Size([2, 6, 4, 40, 40])
Output vertical branch shape: torch.Size([2, 3, 6, 40, 40])
Output surface branch shape: torch.Size([2, 4, 40, 40])


# 3.Train the model

In [ ]:
wandb.finish()
wandb.init(project="Weather_prediction")

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▄▄▅▅▅▅▅▅▅▆▆▆▇▇████▁▁▁▁▂▃▃▃▃▃
test_RMSE_10m_u_component_of_wind,█▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▆▅▅▅▅▅▅▅
test_RMSE_10m_v_component_of_wind,▅▅▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▇▆▅▄▄▄▄▃▃▃▃▃
test_RMSE_2m_temperature,▇▆▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▆▅▅▅▅▄▄
test_RMSE_surface_pressure,█▆▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▅▄▄▃▃▃▃▃
train_loss,█▄▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▃▃▃▃▃
epoch,63
test_RMSE_10m_u_component_of_wind,0.56515
test_RMSE_10m_v_component_of_wind,0.48991
test_RMSE_2m_temperature,0.39902
test_RMSE_surface_pressure,0.1471


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import wandb
import numpy as np

wandb.init(project="mvp_model_training")

seq_length = 6
lead_time = 6
batch_size = 8
num_epochs = 200
learning_rate = 1e-4

surface_vars = ['2m_temperature', '10m_u_component_of_wind', '10m_v_component_of_wind', 'surface_pressure']

# Create full dataset from preprocessed arrays
full_dataset = PanguEraDataset(vertical_data=vert_array, surface_data=surf_array,
                               seq_length=seq_length, lead_time=lead_time)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = MVPModel(
    patch_size=(2, 2, 2),
    in_channels_3d=vert_array.shape[1],
    in_channels_2d=surf_array.shape[1],
    out_channels_3d=vert_array.shape[1],
    out_channels_2d=surf_array.shape[1],
    emb_dim=32,
    num_heads=4
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    num_train_batches = 0
    for batch in train_loader:
        (vert_seq, surf_seq), (vert_target, surf_target) = batch
        vert_seq = vert_seq.to(device)
        surf_seq = surf_seq.to(device)
        vert_target = vert_target.to(device)
        surf_target = surf_target.to(device)
        vert_input = vert_seq[:, -1]
        surf_input = surf_seq[:, -1]
        out_3d, out_2d = model(vert_input, surf_input)
        loss = torch.sqrt(criterion(out_3d, vert_target)) + torch.sqrt(criterion(out_2d, surf_target))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        num_train_batches += 1
    avg_train_loss = running_loss / num_train_batches

    model.eval()
    test_surface_sum_sq = torch.zeros(len(surface_vars), device=device)
    test_surface_total_count = 0
    with torch.no_grad():
        for batch in test_loader:
            (vert_seq, surf_seq), (vert_target, surf_target) = batch
            vert_seq = vert_seq.to(device)
            surf_seq = surf_seq.to(device)
            vert_target = vert_target.to(device)
            surf_target = surf_target.to(device)
            out_3d, out_2d = model(vert_seq[:, -1], surf_seq[:, -1])
            err = (out_2d - surf_target) ** 2
            test_surface_sum_sq += err.sum(dim=(0, 2, 3))
            test_surface_total_count += err.shape[0] * err.shape[2] * err.shape[3]
        test_rmse_per_channel = torch.sqrt(test_surface_sum_sq / test_surface_total_count)

    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}")
    for i, var in enumerate(surface_vars):
        print(f"Test RMSE {var}: {test_rmse_per_channel[i].item():.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "test_RMSE_2m_temperature": test_rmse_per_channel[0].item(),
        "test_RMSE_10m_u_component_of_wind": test_rmse_per_channel[1].item(),
        "test_RMSE_10m_v_component_of_wind": test_rmse_per_channel[2].item(),
        "test_RMSE_surface_pressure": test_rmse_per_channel[3].item()
    })

Epoch [1/200] - Train Loss: 2.2350
Test RMSE 2m_temperature: 0.8961
Test RMSE 10m_u_component_of_wind: 1.1865
Test RMSE 10m_v_component_of_wind: 1.0786
Test RMSE surface_pressure: 1.2153
Epoch [2/200] - Train Loss: 2.0719
Test RMSE 2m_temperature: 0.8418
Test RMSE 10m_u_component_of_wind: 1.1126
Test RMSE 10m_v_component_of_wind: 1.0131
Test RMSE surface_pressure: 1.1126
Epoch [3/200] - Train Loss: 1.9341
Test RMSE 2m_temperature: 0.7962
Test RMSE 10m_u_component_of_wind: 1.0492
Test RMSE 10m_v_component_of_wind: 0.9572
Test RMSE surface_pressure: 1.0246
Epoch [4/200] - Train Loss: 1.8135
Test RMSE 2m_temperature: 0.7559
Test RMSE 10m_u_component_of_wind: 0.9919
Test RMSE 10m_v_component_of_wind: 0.9090
Test RMSE surface_pressure: 0.9448
Epoch [5/200] - Train Loss: 1.7034
Test RMSE 2m_temperature: 0.7181
Test RMSE 10m_u_component_of_wind: 0.9391
Test RMSE 10m_v_component_of_wind: 0.8643
Test RMSE surface_pressure: 0.8711
Epoch [6/200] - Train Loss: 1.6021
Test RMSE 2m_temperature: 0.68

In [ ]:
torch.cuda.empty_cache()

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")

# Evaluate how the model generalizes to October data

In [ ]:
# October evaluation
oct_time_slice = slice("2020-10-01", "2020-10-08")
lat_slice = slice(50, 40)
lon_slice = slice(0, 10)

oct_surf_ds = ds[surface_vars].sel(time=oct_time_slice, latitude=lat_slice, longitude=lon_slice)
oct_vert_ds = ds[vertical_vars].sel(time=oct_time_slice, level=desired_levels, latitude=lat_slice, longitude=lon_slice)

oct_surf_list = []
for var in surface_vars:
    da = oct_surf_ds[var]
    mean_val = da.mean().values
    std_val = da.std().values
    oct_surf_list.append(((da - mean_val) / std_val).values)
oct_surf_array = np.stack(oct_surf_list, axis=1)

oct_vert_list = []
for var in vertical_vars:
    da = oct_vert_ds[var]
    mean_val = da.mean().values
    std_val = da.std().values
    arr = np.expand_dims(((da - mean_val) / std_val).values, axis=1)
    oct_vert_list.append(arr)
oct_vert_array = np.concatenate(oct_vert_list, axis=1)

oct_dataset = PanguEraDataset(vertical_data=oct_vert_array, surface_data=oct_surf_array,
                              seq_length=seq_length, lead_time=lead_time)
oct_loader = DataLoader(oct_dataset, batch_size=batch_size, shuffle=False)

model.eval()
oct_surface_sum_sq = torch.zeros(len(surface_vars), device=device)
oct_surface_total_count = 0
with torch.no_grad():
    for batch in oct_loader:
        (vert_seq, surf_seq), (_, surf_target) = batch
        vert_seq = vert_seq.to(device)
        surf_seq = surf_seq.to(device)
        surf_target = surf_target.to(device)
        _, out_2d = model(vert_seq[:, -1], surf_seq[:, -1])
        err = (out_2d - surf_target) ** 2
        oct_surface_sum_sq += err.sum(dim=(0, 2, 3))
        oct_surface_total_count += err.shape[0] * err.shape[2] * err.shape[3]
oct_rmse_per_channel = torch.sqrt(oct_surface_sum_sq / oct_surface_total_count)
for i, var in enumerate(surface_vars):
    print(f"October RMSE {var}: {oct_rmse_per_channel[i].item():.4f}")